### This notebook is heavily based on the amazing notebook by @aerdem4: https://www.kaggle.com/code/aerdem4/foursquare-rapids-xgb-inference
### I have implemented Spatial KDTree to get the nearest neighbours and it gives us more accuracy

In [ ]:
import cudf

df = cudf.read_csv("../input/foursquare-location-matching/test.csv")
# df = cudf.read_csv("../input/foursquare-location-matching/train.csv").drop(["point_of_interest"],axis = 1)
print(df.shape)
df.head()

In [ ]:
import pandas as pd
import numpy as np
from scipy import spatial

In [ ]:
# # increase data size for debug mode
# if df.shape[0] == 5:
#     df = cudf.concat([df]*10).reset_index(drop=True)
    
# df.shape

In [ ]:
lats = df['latitude'].to_arrow().to_pylist()
lons = df['longitude'].to_arrow().to_pylist()
Z = tuple(zip(lats, lons))
tree = spatial.KDTree(Z)
if df.shape[0] < 100:
    loc_data = tree.query(Z,5)
else:
    loc_data = tree.query(Z,55)    
distances, indices = loc_data

In [ ]:
# from cuml.neighbors import NearestNeighbors

# coo_cols = ["latitude", "longitude"]
# if df.shape[0]==50:
#     matcher = NearestNeighbors(n_neighbors=5)
# else:
#     matcher = NearestNeighbors(n_neighbors=100)
# matcher.fit(df[coo_cols])


# distances, indices = matcher.kneighbors(df[coo_cols])

In [ ]:
from cuml.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
V_name = tfidf.fit_transform(df["name"].fillna("noname"))
V_name.shape

In [ ]:
tfidf = TfidfVectorizer()
V_cat = tfidf.fit_transform(df["categories"].fillna("nocategory"))
V_cat.shape

In [ ]:
from cuml import ForestInference


THRESHOLD = 0.4
features = ["dist", "name_sim", "cat_sim"]


xgb_model = ForestInference.load(f"../input/fs-xgb-public-models/xgb_fs.json", output_class=True, model_type="xgboost_json")

In [ ]:
dfs = []


for i in range(indices.shape[1]):
    tmp_df = df[["id"]].copy()
    
    tmp_df["dist"] = distances[:, i]
    tmp_df["name_sim"] = V_name.multiply(V_name[indices[:, i]]).sum(axis=1).ravel()
    tmp_df["cat_sim"] = V_cat.multiply(V_cat[indices[:, i]]).sum(axis=1).ravel()
    tmp_df["match_id"] = df["id"].to_pandas().values[indices[:, i]]
    
    tmp_df["pred"] = xgb_model.predict_proba(tmp_df[features].to_pandas())[:, 1]
    
    dfs.append(tmp_df[tmp_df["pred"] > THRESHOLD])
    
out_df = cudf.concat(dfs)
out_df.shape

In [ ]:
out_df = df[["id"]].drop_duplicates().to_pandas().append(out_df.to_pandas())

out_df.loc[out_df["match_id"].isnull(), "match_id"] = out_df.loc[out_df["match_id"].isnull(), "id"]

out_df = out_df.groupby("id")["match_id"].apply(list).reset_index()
out_df["matches"] = out_df["match_id"].apply(lambda x: " ".join(set(x)))

out_df.head()

In [ ]:
out_df.to_csv("submission.csv", index=False, columns=["id", "matches"])